DATASET EXTRACTION & PREPROCESSING

In [ ]:
import os
import zipfile
import shutil
import numpy as np
import librosa
from google.colab import drive
from tqdm import tqdm # Loading bar

# --- 1. SAMBUNGKAN GOOGLE DRIVE ---
drive.mount('/content/drive')

# --- KONFIGURASI PATH (Sesuai Screenshot Kamu) ---
# Lokasi folder di Drive kamu
drive_folder = '/content/drive/MyDrive/datasetrnn1'

# Nama file zip persis sesuai di screenshot
zip_files = {
    'clean': 'clean_trainset_28spk_wav.zip',
    'noisy': 'noisy_trainset_28spk_wav.zip'
}

# Lokasi sementara di Colab (Biar cepat prosesnya)
local_extract_path = '/content/data_raw'
processed_path = '/content/data_ready'

# --- 2. COPY & EXTRACT (Handling Multiple Zips) ---
if not os.path.exists(local_extract_path):
    os.makedirs(local_extract_path)

for key, zip_name in zip_files.items():
    source = os.path.join(drive_folder, zip_name)
    dest = os.path.join('/content', zip_name) # Copy ke root colab dulu

    print(f"🚀 Sedang meng-copy {zip_name} dari Drive... (Sabar ya)")

    # Cek apakah file ada di Drive
    if not os.path.exists(source):
        print(f"❌ ERROR: File {zip_name} tidak ditemukan di {drive_folder}!")
        print("   Pastikan nama folder di Drive benar-benar 'datasetrnn1' (huruf kecil semua)")
        raise FileNotFoundError("File zip tidak ketemu.")

    shutil.copy(source, dest)

    print(f"📂 Sedang mengekstrak {zip_name}...")
    with zipfile.ZipFile(dest, 'r') as zip_ref:
        zip_ref.extractall(local_extract_path)

    # Hapus zip sisa biar hemat storage Colab
    os.remove(dest)

print("✅ Semua ekstrak selesai!")

# --- 3. FUNGSI PREPROCESSING (Downsample 48k -> 16k) ---
def process_audio_folder(folder_path, n_fft=512, hop_length=256):
    # Cari file wav di dalam folder (termasuk subfolder jika ada)
    wav_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))

    wav_files.sort() # Urutkan biar clean & noisy sinkron (p226_001, p226_002, ...)
    data_list = []

    print(f"🔄 Memproses {len(wav_files)} file dari: {folder_path}")
    print("   (Loading audio, Resampling 48k->16k, STFT...)")

    # Kita batasi dulu untuk tes (misal 500 file pertama) biar RAM ga meledak
    # Kalau RAM kuat, hapus [:2000] nya.
    for path in tqdm(wav_files):
        try:
            # PENTING: sr=16000 memaksa Python mengecilkan sample rate dari 48k
            y, sr = librosa.load(path, sr=16000)

            # STFT (Time-Frequency transform)
            S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)

            # Ambil Magnitude (Input ke Neural Network)
            S_mag = np.abs(S)

            # Transpose: (Freq, Time) -> (Time, Freq)
            data_list.append(S_mag.T)
        except Exception as e:
            print(f"⚠️ Gagal memproses {path}: {e}")

    return data_list

# --- 4. CARI FOLDER HASIL EKSTRAK ---
# Kadang hasil ekstrak ada di dalam subfolder, kita cari otomatis
all_dirs = [x[0] for x in os.walk(local_extract_path)]
clean_folder = next((d for d in all_dirs if 'clean_trainset' in d), None)
noisy_folder = next((d for d in all_dirs if 'noisy_trainset' in d), None)

if not clean_folder or not noisy_folder:
    print("❌ Error: Folder hasil ekstrak tidak ditemukan. Cek struktur zip.")
else:
    print(f"📍 Target Clean: {clean_folder}")
    print(f"📍 Target Noisy: {noisy_folder}")

    # --- 5. EKSEKUSI ---
    clean_data = process_audio_folder(clean_folder)
    noisy_data = process_audio_folder(noisy_folder)

    # --- 6. SIMPAN DATA MATANG ---
    os.makedirs(processed_path, exist_ok=True)

    # Simpan sebagai object array (karena durasi audio beda-beda)
    print("💾 Menyimpan file .npy...")
    np.save(f'{processed_path}/clean_spectrograms.npy', np.array(clean_data, dtype=object))
    np.save(f'{processed_path}/noisy_spectrograms.npy', np.array(noisy_data, dtype=object))

    print("\n🎉 SIAP! Data tersimpan di folder 'data_ready' di sebelah kiri.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Sedang meng-copy clean_trainset_28spk_wav.zip dari Drive... (Sabar ya)
📂 Sedang mengekstrak clean_trainset_28spk_wav.zip...
🚀 Sedang meng-copy noisy_trainset_28spk_wav.zip dari Drive... (Sabar ya)
📂 Sedang mengekstrak noisy_trainset_28spk_wav.zip...
✅ Semua ekstrak selesai!
📍 Target Clean: /content/data_raw/clean_trainset_28spk_wav
📍 Target Noisy: /content/data_raw/noisy_trainset_28spk_wav
🔄 Memproses 11572 file dari: /content/data_raw/clean_trainset_28spk_wav
   (Loading audio, Resampling 48k->16k, STFT...)


100%|██████████| 11572/11572 [01:09<00:00, 166.16it/s]


🔄 Memproses 11572 file dari: /content/data_raw/noisy_trainset_28spk_wav
   (Loading audio, Resampling 48k->16k, STFT...)


100%|██████████| 11572/11572 [01:08<00:00, 169.63it/s]


💾 Menyimpan file .npy...

🎉 SIAP! Data tersimpan di folder 'data_ready' di sebelah kiri.
   Sekarang kamu bisa lanjut bikin Model LSTM.


MODEL TRAINING (LSTM)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
import numpy as np

# --- 1. KONFIGURASI ---
BATCH_SIZE = 32
MAX_LEN = 128  # Panjang potongan audio (sekitar 1 detik biar enteng)
EPOCHS = 30    # Jumlah putaran latihan

# --- 2. CEK DATA DI RAM ---
if 'clean_data' not in locals() or 'noisy_data' not in locals():
    print("❌ Eits! Variabel data hilang. Pastikan kamu sudah run script preprocessing sebelumnya (tanpa save).")
else:
    print(f"✅ Data siap! Clean: {len(clean_data)}, Noisy: {len(noisy_data)}")

# --- 3. SPLIT DATA (Latih vs Ujian) ---
# Kita sisihkan 20% data buat Validasi (biar tau modelnya jago beneran apa cuma ngafal)
X_train, X_val, y_train, y_val = train_test_split(noisy_data, clean_data, test_size=0.2, random_state=42)
print(f"📊 Training Set: {len(X_train)} sampel")
print(f"📊 Validation Set: {len(X_val)} sampel")

# --- 4. DATA GENERATOR (Penyelamat RAM) ---
# Fungsi ini ngasih makan model sedikit-sedikit, bukan sekaligus
def data_generator(noisy_list, clean_list, batch_size=32):
    while True:
        X_batch, y_batch = [], []

        # Ambil index acak
        indices = np.random.randint(0, len(noisy_list), batch_size)

        for idx in indices:
            n_spec = noisy_list[idx]
            c_spec = clean_list[idx]

            # Potong audio secara acak biar panjangnya seragam (MAX_LEN)
            # Ini teknik "Random Cropping" -> Bagus buat variasi training
            if n_spec.shape[0] > MAX_LEN:
                start = np.random.randint(0, n_spec.shape[0] - MAX_LEN)
                n_cut = n_spec[start : start + MAX_LEN]
                c_cut = c_spec[start : start + MAX_LEN]

                X_batch.append(n_cut)
                y_batch.append(c_cut)

        if len(X_batch) > 0:
            yield np.array(X_batch), np.array(y_batch)

# --- 5. BANGUN ARSITEKTUR MODEL (LSTM) ---
def build_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Layer 1: LSTM (256 unit) - Otak utama
    x = layers.LSTM(256, return_sequences=True)(inputs)
    x = layers.Dropout(0.2)(x) # Biar ga overfitting

    # Layer 2: LSTM (256 unit) - Otak kedua biar makin pinter
    x = layers.LSTM(256, return_sequences=True)(x)
    x = layers.Dropout(0.2)(x)

    # Output Layer: Mengembalikan ke bentuk spectrogram asli (257 frekuensi)
    # Kita pakai 'relu' karena magnitude suara nggak mungkin negatif
    outputs = layers.TimeDistributed(layers.Dense(257, activation='relu'))(x)

    model = models.Model(inputs=inputs, outputs=outputs)

    # Compile: Pakai 'mae' (Mean Absolute Error) yang lebih robust buat audio
    model.compile(optimizer='adam', loss='mae')
    return model

# Input shape: (Waktu, Frekuensi 257 bin)
model = build_model((MAX_LEN, 257))
model.summary()

# --- 6. CALLBACKS (Robot Penjaga) ---
my_callbacks = [
    # Kalau validation loss ga turun selama 5 epoch, stop aja (hemat waktu)
    callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss', verbose=1),
    # Simpan model terbaik secara otomatis
    callbacks.ModelCheckpoint('model_denoiser_best.h5', save_best_only=True, monitor='val_loss', verbose=1)
]

# --- 7. MULAI TRAINING! 🔥 ---
print("\n🚀 Mesin menyala! Training dimulai...")
history = model.fit(
    data_generator(X_train, y_train, BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    validation_data=data_generator(X_val, y_val, BATCH_SIZE),
    validation_steps=len(X_val) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=my_callbacks
)

print("\n🎉 Training Selesai! Model terbaik tersimpan sebagai 'model_denoiser_best.h5'")

✅ Data siap! Clean: 11572, Noisy: 11572
📊 Training Set: 9257 sampel
📊 Validation Set: 2315 sampel


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 257)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128, 256)       │       526,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128, 256)       │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 128, 257)       │        66,049 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,117,697 (4.26 MB)

 Trainable params: 1,117,697 (4.26 MB)

...

289/289 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - loss: 0.0755 - val_loss: 0.0650
Epoch 22/30
289/289 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0755
Epoch 22: val_loss did not improve from 0.06500
289/289 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - loss: 0.0755 - val_loss: 0.0658
Epoch 23/30
288/289 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0750
Epoch 23: val_loss did not improve from 0.06500
289/289 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - loss: 0.0750 - val_loss: 0.0650
Epoch 24/30
288/289 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0748
Epoch 24: val_loss improved from 0.06500 to 0.06454, saving model to model_denoiser_best.h5


289/289 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - loss: 0.0748 - val_loss: 0.0645
Epoch 25/30
289/289 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0740
Epoch 25: val_loss improved from 0.06454 to 0.06396, saving model to model_denoiser_best.h5


289/289 ━━━━━━━━━━━━━━━━━━━━ 21s 71ms/step - loss: 0.0740 - val_loss: 0.0640
Epoch 26/30
288/289 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0743
Epoch 26: val_loss did not improve from 0.06396
289/289 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - loss: 0.0743 - val_loss: 0.0651
Epoch 27/30
288/289 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0739
Epoch 27: val_loss did not improve from 0.06396
289/289 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - loss: 0.0739 - val_loss: 0.0650
Epoch 28/30
289/289 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0734
Epoch 28: val_loss improved from 0.06396 to 0.06376, saving model to model_denoiser_best.h5


289/289 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - loss: 0.0734 - val_loss: 0.0638
Epoch 29/30
289/289 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0731
Epoch 29: val_loss improved from 0.06376 to 0.06348, saving model to model_denoiser_best.h5


289/289 ━━━━━━━━━━━━━━━━━━━━ 21s 72ms/step - loss: 0.0731 - val_loss: 0.0635
Epoch 30/30
288/289 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0736
Epoch 30: val_loss did not improve from 0.06348
289/289 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - loss: 0.0736 - val_loss: 0.0636
Restoring model weights from the end of the best epoch: 29.

🎉 Training Selesai! Model terbaik tersimpan sebagai 'model_denoiser_best.h5'


In [ ]:
MODEL BACKUP TO GOOGLE DRIVE

In [ ]:
import shutil

# Simpan ke Google Drive biar aman abadi
# Pastikan path folder Drive-nya benar ya
shutil.copy('model_denoiser_best.h5', '/content/drive/MyDrive/datasetrnn1/MODEL_JADI_LSTM.h5')

print("✅ Model berhasil diamankan ke Google Drive!")

✅ Model berhasil diamankan ke Google Drive!
